![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=500>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

### The Structure of a SQL Query

<img src='img/sql_statement.jpg'/>

***
## Goal 3: Get going with sqlite!

In [102]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [103]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [104]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [105]:
cursor.description

(('name', None, None, None, None, None, None),)

In [106]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [107]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [108]:
cursor.execute("SELECT * FROM airports").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [109]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [110]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [111]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [112]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'city', 'TEXT', 0, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'code', 'TEXT', 0, None, 0),
 (6, 'icao', 'TEXT', 0, None, 0),
 (7, 'latitude', 'TEXT', 0, None, 0),
 (8, 'longitude', 'TEXT', 0, None, 0),
 (9, 'altitude', 'TEXT', 0, None, 0),
 (10, 'offset', 'TEXT', 0, None, 0),
 (11, 'dst', 'TEXT', 0, None, 0),
 (12, 'timezone', 'TEXT', 0, None, 0)]

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [113]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'alias', 'TEXT', 0, None, 0),
 (4, 'iata', 'TEXT', 0, None, 0),
 (5, 'icao', 'TEXT', 0, None, 0),
 (6, 'callsign', 'TEXT', 0, None, 0),
 (7, 'country', 'TEXT', 0, None, 0),
 (8, 'active', 'TEXT', 0, None, 0)]

In [114]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'airline', 'TEXT', 0, None, 0),
 (2, 'airline_id', 'TEXT', 0, None, 0),
 (3, 'source', 'TEXT', 0, None, 0),
 (4, 'source_id', 'TEXT', 0, None, 0),
 (5, 'dest', 'TEXT', 0, None, 0),
 (6, 'dest_id', 'TEXT', 0, None, 0),
 (7, 'codeshare', 'TEXT', 0, None, 0),
 (8, 'stops', 'TEXT', 0, None, 0),
 (9, 'equipment', 'TEXT', 0, None, 0)]

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [115]:
cursor.execute("""
SELECT *
FROM airlines
""")
cursor.fetchmany(5)

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]

In [116]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [139]:
cursor.execute("""
SELECT routes. *, airports.longitude, airports.latitude FROM routes LEFT JOIN airports ON routes.dest=airports.code
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('airline', None, None, None, None, None, None)
('airline_id', None, None, None, None, None, None)
('source', None, None, None, None, None, None)
('source_id', None, None, None, None, None, None)
('dest', None, None, None, None, None, None)
('dest_id', None, None, None, None, None, None)
('codeshare', None, None, None, None, None, None)
('stops', None, None, None, None, None, None)
('equipment', None, None, None, None, None, None)
('longitude', None, None, None, None, None, None)
('latitude', None, None, None, None, None, None)


In [119]:
cursor.execute("""
SELECT * FROM airports
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('id', None, None, None, None, None, None)
('name', None, None, None, None, None, None)
('city', None, None, None, None, None, None)
('country', None, None, None, None, None, None)
('code', None, None, None, None, None, None)
('icao', None, None, None, None, None, None)
('latitude', None, None, None, None, None, None)
('longitude', None, None, None, None, None, None)
('altitude', None, None, None, None, None, None)
('offset', None, None, None, None, None, None)
('dst', None, None, None, None, None, None)
('timezone', None, None, None, None, None, None)


In [17]:
cursor.execute("""
SELECT * FROM routes 
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('airline', None, None, None, None, None, None)
('airline_id', None, None, None, None, None, None)
('source', None, None, None, None, None, None)
('source_id', None, None, None, None, None, None)
('dest', None, None, None, None, None, None)
('dest_id', None, None, None, None, None, None)
('codeshare', None, None, None, None, None, None)
('stops', None, None, None, None, None, None)
('equipment', None, None, None, None, None, None)


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [18]:
cursor.execute("""
SELECT * FROM airlines 

""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('id', None, None, None, None, None, None)
('name', None, None, None, None, None, None)
('alias', None, None, None, None, None, None)
('iata', None, None, None, None, None, None)
('icao', None, None, None, None, None, None)
('callsign', None, None, None, None, None, None)
('country', None, None, None, None, None, None)
('active', None, None, None, None, None, None)


In [19]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
SELECT country, count(*) as c
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY c DESC;

""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)
('Japan', 19)
('Italy', 18)
('Indonesia', 17)
('India', 17)
('Turkey', 16)
('Thailand', 16)
('Sweden', 15)
('Switzerland', 14)
('Ukraine', 13)
('Portugal', 13)
('Mexico', 12)
('Finland', 12)
('Egypt', 12)
('Austria', 12)
('Peru', 11)
('South Africa', 10)
('Greece', 10)
('Argentina', 10)
('Netherlands', 9)
('Iceland', 9)
('Colombia', 9)
('United Arab Emirates', 8)
('Singapore', 8)
('Romania', 8)
('Poland', 8)
('Philippines', 8)
('Vietnam', 7)
('Taiwan', 7)
('Pakistan', 7)
('Malaysia', 7)
('Ireland', 7)
('Iran', 7)
('Ecuador', 7)
('Denmark', 7)
('South Korea', 6)
('Norway', 6)
('Nepal', 6)
('Kazakhstan', 6)
('Israel', 6)
('Georgia', 6)
('Chile', 6)
('Burma', 6)
('Belgium', 6)
('Slovakia', 5)
('Republic of Korea', 5)
('Nigeria', 5)
('Morocco', 5)
('Lithuania', 5)
('Dominican Republic', 5)
('Czech Republic', 5)
('Cambodia', 

In [20]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
SELECT country, count(*) as c
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY c DESC;
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)
('Spain', 142)
('France', 97)
('Germany', 94)
('South Africa', 81)
('Nigeria', 80)
('Ukraine', 76)
('Kazakhstan', 73)
('Italy', 72)
('Australia', 67)
('Sweden', 55)
('Sudan', 47)
('Switzerland', 46)
('China', 45)
('Netherlands', 43)
('Austria', 38)
('Egypt', 36)
('Chile', 35)
('Brazil', 35)
('Angola', 35)
('Colombia', 34)
('Thailand', 32)
('Portugal', 32)
('Mauritania', 32)
('Indonesia', 31)
('Kyrgyzstan', 29)
('Czech Republic', 27)
('Japan', 25)
('Dominican Republic', 25)
('Denmark', 25)
('Bulgaria', 25)
('New Zealand', 24)
('Iran', 24)
('United Arab Emirates', 23)
('Turkey', 23)
('Sierra Leone', 23)
('Norway', 23)
('Libya', 23)
('Venezuela', 22)
('Pakistan', 22)
('Uganda', 21)
('Ireland', 21)
('Belgium', 21)
('Zambia', 20)
('Kenya', 20)
('Greece', 19)
('Ecuador', 19)
('Democratic Republic of the Congo', 19)
('Moldova', 17)
('Ghana', 17)
('Israel', 16)
('Peru', 15)
('Ivory Coast', 15)
('Hung

In [21]:
# What about airports by timezones?
# grouping the airports by timezone 
cursor.execute("""
SELECT name, timezone, count(*) as c
FROM airports
GROUP BY timezone
ORDER BY c DESC;
""")
print(*cursor.fetchall(), sep='\n')

('Putnam County Airport', 'America/New_York', 628)
('Door County Cherryland Airport', 'America/Chicago', 373)
('Bautzen', 'Europe/Berlin', 319)
('Tyonek Airport', 'America/Anchorage', 258)
('Calais Dunkerque', 'Europe/Paris', 232)
('Eastern Oregon Regional Airport', 'America/Los_Angeles', 226)
('Capital Intl', 'Asia/Chongqing', 222)
('Belfast Intl', 'Europe/London', 193)
('Sault Ste Marie', 'America/Toronto', 159)
('Ahmedabad', 'Asia/Calcutta', 141)
('Riverton Regional', 'America/Denver', 135)
('Narita Intl', 'Asia/Tokyo', 131)
('Aeropuerto de Rafaela', 'America/Cordoba', 104)
('Alexander Bay', 'Africa/Johannesburg', 104)
('Pulkovo', 'Europe/Moscow', 100)
('Campo Delio Jardim De Mattos', 'America/Sao_Paulo', 97)
('Amendola', 'Europe/Rome', 93)
('Guantanamo Bay Ns', '\\N', 92)
('Malmen', 'Europe/Stockholm', 86)
('Whitsunday Airstrip', 'Australia/Brisbane', 81)
('Abadan', 'Asia/Tehran', 81)
('Tofino', 'America/Vancouver', 78)
('Guvercinlik', 'Europe/Istanbul', 77)
('Melilla', 'Europe/Mad

In [22]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Transfering from sqlite to pandas

In [141]:
import pandas as pd

pd_con = sqlite3.connect("data/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [142]:
# Which countries have the highest amount of active airports?
sql = '''
SELECT a1.country, a1.name, count(active) as active_airlines
FROM airports as a1 JOIN airlines as a2 ON a1.icao = a2.icao
WHERE active = 'Y'
GROUP BY a1.country
ORDER BY active_airlines DESC;

'''
pd.read_sql_query(sql, pd_con)

,country,name,active_airlines
0,United States,Mercer County Airport,45222
1,Germany,Aachen HBF,7735
2,Canada,Albian Aerodrome,7259
3,Russia,Angasolka,6545
4,China,All Airports,6545
...,...,...,...
123,Azerbaijan,Heydar Aliyev,119
124,Angola,Catoca Airport,119
125,American Samoa,Ofu Airport,119
126,Algeria,El Merk,119


In [25]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [143]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()
import pandas as pd

In [144]:
# How many tables are in the database?

# your code here

sql = '''
SELECT count(tbl_name) FROM sqlite_master AS tables 
WHERE type='table';

'''
table_name=pd.read_sql_query(sql, conn)
table_name

,count(tbl_name)
0,11


In [126]:
# What's the primary key of each table?

# your code here

sql = '''SELECT l.name FROM pragma_table_info('Album') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('Artist') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('Customer') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('Employee') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('Genre') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('Invoice') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('InvoiceLine') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('MediaType') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('Playlist') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('PlaylistTrack') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

sql = '''SELECT l.name FROM pragma_table_info('Track') as l WHERE l.pk = 1;'''
print(pd.read_sql_query(sql, conn))

      name
0  AlbumId
       name
0  ArtistId
         name
0  CustomerId
         name
0  EmployeeId
      name
0  GenreId
        name
0  InvoiceId
            name
0  InvoiceLineId
          name
0  MediaTypeId
         name
0  PlaylistId
         name
0  PlaylistId
      name
0  TrackId


Foreign keys for playlistTrack table is TrackID and PlyalistID

In [145]:
sql = '''
SELECT * FROM pragma_foreign_key_list('Album')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,Artist,ArtistId,ArtistId,NO ACTION,NO ACTION,NONE


In [146]:
sql = '''
SELECT * FROM pragma_foreign_key_list('Artist')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match


In [147]:
sql = '''
SELECT * FROM pragma_foreign_key_list('Customer')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,Employee,SupportRepId,EmployeeId,NO ACTION,NO ACTION,NONE


In [148]:
sql = '''
SELECT * FROM pragma_foreign_key_list('Employee')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,Employee,ReportsTo,EmployeeId,NO ACTION,NO ACTION,NONE


In [149]:
sql = '''
SELECT * FROM pragma_foreign_key_list('Genre')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match


In [150]:
sql = '''
SELECT * FROM pragma_foreign_key_list('Invoice')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,Customer,CustomerId,CustomerId,NO ACTION,NO ACTION,NONE


In [153]:
sql = '''
SELECT * FROM pragma_foreign_key_list('InvoiceLine')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,Track,TrackId,TrackId,NO ACTION,NO ACTION,NONE
1,1,0,Invoice,InvoiceId,InvoiceId,NO ACTION,NO ACTION,NONE


In [154]:
sql = '''
SELECT * FROM pragma_foreign_key_list('MediaType')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match


In [155]:
sql = '''
SELECT * FROM pragma_foreign_key_list('Playlist')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match


In [151]:
sql = '''
SELECT * FROM pragma_foreign_key_list('playlistTrack')
'''
pd.read_sql_query(sql, conn)



,id,seq,table,from,to,on_update,on_delete,match
0,0,0,Track,TrackId,TrackId,NO ACTION,NO ACTION,NONE
1,1,0,Playlist,PlaylistId,PlaylistId,NO ACTION,NO ACTION,NONE


In [152]:
sql = '''
SELECT * FROM pragma_foreign_key_list('Track')
'''
pd.read_sql_query(sql, conn)

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,MediaType,MediaTypeId,MediaTypeId,NO ACTION,NO ACTION,NONE
1,1,0,Genre,GenreId,GenreId,NO ACTION,NO ACTION,NONE
2,2,0,Album,AlbumId,AlbumId,NO ACTION,NO ACTION,NONE


In [128]:
# What are the max and min dates in the Invoice table?
sql = '''
SELECT MAX(InvoiceDate), MIN(InvoiceDate) FROM Invoice

'''
pd.read_sql_query(sql, conn)

,MAX(InvoiceDate),MIN(InvoiceDate)
0,2013-12-22 00:00:00,2009-01-01 00:00:00


#### What tables would you need to answer "what is your most popular track?"

1. PlaylistTrack
2. Track

####  What values from each table?
1. PlaylistTrack
    - playlistID
    - trackID
2. Track
    - TrackID
    - Name
    

In [167]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

sql = '''

SELECT count(DISTINCT p.playlistid) as count, p.trackid, t.name from playlisttrack as p

JOIN track as t

ON t.trackid = p.trackid

GROUP BY p.trackid
ORDER BY count DESC LIMIT 10

'''
pd.read_sql_query(sql, conn)

,count,TrackId,Name
0,5,3403,Intoitus: Adorate Deum
1,5,3404,"Miserere mei, Deus"
2,5,3408,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V..."
3,5,3409,"Suite for Solo Cello No. 1 in G Major, BWV 100..."
4,5,3410,"The Messiah: Behold, I Tell You a Mystery... T..."
5,5,3411,Solomon HWV 67: The Arrival of the Queen of Sheba
6,5,3415,Symphony No.5 in C Minor: I. Allegro con brio
7,5,3416,Ave Maria
8,5,3417,"Nabucco: Chorus, ""Va, Pensiero, Sull'ali Dorate"""
9,5,3418,Die Walküre: The Ride of the Valkyries


In [169]:
# Advanced: get the artist who sang the song!
sql = '''

SELECT a2.name, count(DISTINCT p.playlistid) as count, p.trackid, t.name from playlisttrack as p

JOIN track as t

ON t.trackid = p.trackid

JOIN Album as a1
ON t.albumid = a1.albumid

JOIN Artist as a2
ON a2.artistid = a1.artistid

GROUP BY p.trackid
ORDER BY count DESC LIMIT 10

'''
pd.read_sql_query(sql, conn)

,Name,count,TrackId,Name
0,Alberto Turco & Nova Schola Gregoriana,5,3403,Intoitus: Adorate Deum
1,"Richard Marlow & The Choir of Trinity College,...",5,3404,"Miserere mei, Deus"
2,Wilhelm Kempff,5,3408,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V..."
3,Yo-Yo Ma,5,3409,"Suite for Solo Cello No. 1 in G Major, BWV 100..."
4,Scholars Baroque Ensemble,5,3410,"The Messiah: Behold, I Tell You a Mystery... T..."
5,Academy of St. Martin in the Fields & Sir Nevi...,5,3411,Solomon HWV 67: The Arrival of the Queen of Sheba
6,Orchestre Révolutionnaire et Romantique & John...,5,3415,Symphony No.5 in C Minor: I. Allegro con brio
7,"Britten Sinfonia, Ivor Bolton & Lesley Garrett",5,3416,Ave Maria
8,"Chicago Symphony Chorus, Chicago Symphony Orch...",5,3417,"Nabucco: Chorus, ""Va, Pensiero, Sull'ali Dorate"""
9,Sir Georg Solti & Wiener Philharmoniker,5,3418,Die Walküre: The Ride of the Valkyries


#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_alexes
FROM student_courses
WHERE student_name = "Alex"
GROUP BY 1
HAVING COUNT(student_name) >= 2;
```

In [171]:
conn = sqlite3.connect('data/tutorial.db')
c = conn.cursor()

import pandas as pd

In [172]:
sql = '''
SELECT tbl_name FROM sqlite_master AS tables 
WHERE type='table';

'''
table_name=pd.read_sql_query(sql, conn)
table_name

,tbl_name
0,responses


In [173]:
columns = [x[0] for x in c.execute('select name from responses').description]

In [174]:
pd_con = sqlite3.connect('data/tutorial.db')
df = pd.read_sql_query("select * from responses limit 5;", pd_con)
df

,submitted_ts,name,birthday,hometown,fav_food_1,fav_food_2,time_in_dc,siblings_count
0,12/11/2019 12:57:49,Stephen,12/8,"Arlington, TX",pizza,beef stroganoff,0.0,2
1,12/11/2019 13:04:35,Donna C,9/29,"Phoenix, AZ",shrimp,sushi,0.2,1
2,12/11/2019 13:05:08,Muoyo,6/22,"Brooklyn, NY",Nigerian,Thai,0.0,1
3,12/11/2019 13:06:13,Vyjayanthi,7/24,"Fairfax, VA",Pasta,Noodles,11.0,3
4,12/11/2019 13:21:20,Anesu Masube,3/28,"Kwekwe, Zimbabwe",Pork Ribs,Sadza,3.0,5


### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

In [175]:
sql = '''
SELECT name FROM responses 
'''
pd.read_sql_query(sql, conn)

,name
0,Stephen
1,Donna C
2,Muoyo
3,Vyjayanthi
4,Anesu Masube
5,Michael Pallante
6,Stuart Murphy
7,Darian Madere
8,Alex
9,Justin Fleury


2. Which student has the most siblings?

In [176]:
sql = '''
SELECT submitted_ts,name,birthday,hometown,MAX(siblings_count) FROM responses AS Most_Siblings
'''
pd.read_sql_query(sql, conn)

,submitted_ts,name,birthday,hometown,MAX(siblings_count)
0,12/11/2019 13:58:12,Nick,11/1,"Colorado Springs, CO",12


3. How many students are only children?

In [177]:
sql = '''
SELECT * FROM responses WHERE siblings_count=0
'''
pd.read_sql_query(sql, conn)

,submitted_ts,name,birthday,hometown,fav_food_1,fav_food_2,time_in_dc,siblings_count


4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [178]:
#Which 3 students have lived in DC the shortest amount of time? (How long has each lived in DC?)
sql = '''
SELECT * FROM responses WHERE time_in_dc != 0.0
ORDER BY time_in_dc
LIMIT 3;
'''
pd.read_sql_query(sql, conn)

,submitted_ts,name,birthday,hometown,fav_food_1,fav_food_2,time_in_dc,siblings_count
0,12/11/2019 13:04:35,Donna C,9/29,"Phoenix, AZ",shrimp,sushi,0.2,1
1,12/11/2019 13:38:17,Michael Pallante,8/5,"Voorhees, NJ",Pizza,Pasta,1.0,2
2,12/11/2019 13:48:58,Darian Madere,8/7,"Baton Ruuge, LA",tater tots,sushi,1.0,3


5. How many students are native New Yorkers?

In [179]:
sql = '''
SELECT * FROM responses WHERE hometown LIKE '%NY'
'''
pd.read_sql_query(sql, conn)

,submitted_ts,name,birthday,hometown,fav_food_1,fav_food_2,time_in_dc,siblings_count
0,12/11/2019 13:05:08,Muoyo,6/22,"Brooklyn, NY",Nigerian,Thai,0.0,1
1,12/11/2019 14:11:11,Nimu,1/19,"Nyack, NY",Saag,Zucchini,1.5,1


6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [186]:
sql = '''

SELECT * FROM responses 
GROUP BY name

'''
pd.read_sql_query(sql, conn)

,submitted_ts,name,birthday,hometown,fav_food_1,fav_food_2,time_in_dc,siblings_count
0,12/11/2019 13:49:07,Alex,2/21,"Alexandria, VA",Ice cream,Carrot cake,3.0,1
1,12/11/2019 13:21:20,Anesu Masube,3/28,"Kwekwe, Zimbabwe",Pork Ribs,Sadza,3.0,5
2,12/11/2019 14:07:10,Chuck,3/12,"Los Angeles, CA",Crab cakes,Lobster,1.0,3
3,12/11/2019 13:48:58,Darian Madere,8/7,"Baton Ruuge, LA",tater tots,sushi,1.0,3
4,12/11/2019 13:04:35,Donna C,9/29,"Phoenix, AZ",shrimp,sushi,0.2,1
5,12/11/2019 14:00:48,Jeff Lin,4/30,"Los Angeles, CA",Dark Chocolate,Coffee,7.0,2
6,12/11/2019 14:07:13,Jill Carrie,11/13,"Victoria, BC",Thai,Sushi,10.0,1
7,12/11/2019 13:52:47,Justin Fleury,5/5,"Washington, DC",lasagna,brownies,1.0,1
8,12/11/2019 13:38:17,Michael Pallante,8/5,"Voorhees, NJ",Pizza,Pasta,1.0,2
9,12/11/2019 13:05:08,Muoyo,6/22,"Brooklyn, NY",Nigerian,Thai,0.0,1


## More Questions

What are the favorite foods of this classroom?

In [185]:
sql = '''
 
 SELECT derivedtable.Food
FROM
(
   SELECT  fav_food_1 as Food FROM responses UNION SELECT fav_food_2 as Food FROM responses
) derivedtable

'''
pd.read_sql_query(sql, conn)

,Food
0,A single poached egg on a bed of arugula
1,Bacon
2,Bread
3,Buffalo Wings
4,Carrot cake
5,Coffee
6,Crab cakes
7,Dark Chocolate
8,Grey
9,Ice cream


Which student was born closest to the cohort's graduation date (5/29/20)?

In [192]:
sql = '''
SELECT * FROM responses WHERE birthday LIKE'5%'
'''
pd.read_sql_query(sql, conn)

,submitted_ts,name,birthday,hometown,fav_food_1,fav_food_2,time_in_dc,siblings_count
0,12/11/2019 13:52:47,Justin Fleury,5/5,"Washington, DC",lasagna,brownies,1.0,1
